# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_P_hf_counts.ipynb

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from obspy import read
from obspy import Stream
from pysep import Pysep
from pysep.recsec import plotw_rs

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

class SEED_ID_Error(Exception):
    pass

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for i, item in enumerate(event):
    print(f'{item} = {event[item]}')
print()

In [ ]:
# download BHZ data - response removed to displacement

stations   = dict( networks                     = 'G,II,IU',
                   stations                     = '*',
                   locations                    = '*',
                   channels                     = 'BHZ' )

screening  = dict( seconds_before_ref           = 1 * 60 * 60,
                   seconds_after_ref            = 4 * 60 * 60,
                   remove_clipped               = False,
                   remove_insufficient_length   = False,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 0.03,
                   resample_freq                = 20     )

processing = dict( remove_response              = False )

output     = dict( log_level                    = 'INFO',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '',
                   overwrite_event_tag          = 'sumatra_P_hf_counts',
                   overwrite                    = False )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('\nData directory already exists, no data will be downloaded\n')

In [ ]:
# read downloaded raw BHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# source station map for all stations in the data download request

event_path = f'{data_dir}/event.xml'
station_map_and_table(st, event_path)

In [ ]:
# plot a record section using PySEP's record section plotting tool
# change the following parameters as needed

plot_info = dict( st                           = st,
                  sort_by                      = 'alphabetical',
                  scale_by                     = 'normalize',
                  time_shift_s                 = None,
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  xlim_s                       = [-50, 7200],
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'DEBUG')

plotw_rs(**plot_info)

In [ ]:
# selecting waveforms with high frequency post-rupture signals

# modify the following list of picked waveforms -
picked_waveforms = ['G.CAN..BHZ'    ,
                    'II.NNA.00.BHZ' ,
                    'II.SUR.00.BHZ' ,
                    'IU.BBSR.00.BHZ']

st_select = Stream()

for seed_id in picked_waveforms:
    st_pick = st.select(id=seed_id)
    if not st_pick:
        raise SEED_ID_Error(f'{seed_id} not in inventory')
    st_select += st_pick

In [ ]:
# plot a record section using PySEP's record section plotting tool
# change the following parameters as needed

plot_info = dict( st                           = st_select,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  time_shift_s                 = None,
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  xlim_s                       = [-50, 7200],
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'DEBUG')

plotw_rs(**plot_info)